In [14]:
import helpers as hp
import nltk
import csv
from nltk.corpus import wordnet as wn
import preprocess 
from preprocess import pos_tag_db, lemmatize_db
import csv
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from re import search
# Get a list of available databases
dbs = hp.getDatabases()

In [15]:
#creating a list of ingredients
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv
import sys
url = "https://world.openfoodfacts.org/ingredients.json"
page = urlopen(url)
html = page.read()
soup = BeautifulSoup(html) 
u = soup.decode('utf-8')
u = u.split('"name":')
#creating a csv file with ingredients
localFile = open('ingredients.csv', 'w')
#creating a list of ingredients
ingredients = []
for line in u:
    ingredients.append(line.split(',')[0])
del ingredients[0]
for ingredient in ingredients:
    localFile.write(ingredient)
localFile.close()


In [16]:
#we have 10000 ingredients in this vegetarian dataset
len(ingredients)

10000

In [17]:
for element in ingredients:
    #remove any string that contains digits, these are the E numbers
    if any(map(str.isdigit, element)):
        ingredients.remove(element)

In [18]:
ingredients_str = str(ingredients)
cleanString1 = re.sub('"','', ingredients_str )
cleanString = re.sub("'",'', cleanString1)
ingredients = cleanString
ingredients = ingredients.split(',')

In [19]:
for ingr in ingredients:
    if len(ingr) < 4:
        ingredients.remove(ingr)
final = []        
for ingr in ingredients:
    final.append(ingr[1:])
ingredients = final

In [20]:
saved = ["egg", "fat", "oat", "ham", "tea", "ham", "oil","pea", "rye", "fig", "cod", "ice"]
to_remove = [ 'green', 't', 'cone', 'glaze','ngredients','gredients', 'powder', 'sauce','Sauce' , 'serving', 'whole', 'brown','ingredient', 'flakes', 'maple', 'baking', 'serv', 'black', 'diced', 'white', 'paste', 'roll', 'cooked', 'blend', 'approx', 'cooked', 'ingredients', 'dehydrated']
for ingr in ingredients:
    if ingr in to_remove:
        ingredients.remove(ingr)
    if len(ingr) == 3 and ingr.lower() not in saved:
        ingredients.remove(ingr)
   
        
    

## First database : 'VegetarianRecipes.exl'

In [21]:
#data structure of clean_db is a list of strings
clean_db = hp.cleanFile(dbs[0])
  
#merging the cleaned sentenes 
vegetarian_recipes = []
for sentence in clean_db:
    vegetarian_recipes.append(sentence)
#split into separate recipes
split_vegetarian_recipes = str(vegetarian_recipes).split('EACH')
del split_vegetarian_recipes[0]

In [22]:
#writing the recipes to a csv file
with open('recipes_veg.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["EACH PORTION", 'TEMPERATURE', "METHODstr", "METHOD", "NOTE", "CONTAINS (NOUNS)", "INGREDIENTS"])
    #loop through each recipes 
    allmethod = []
    for recipe in split_vegetarian_recipes:
        #for each recipe create a list
        portion = []
        method = []
        temperature = []
        pan_size = []
        note = []
        contains = []
        for sent in re.split(',|\n', recipe):
            if search('PORTION:', sent):
                sent.replace('PORTION:','')
                sent.replace('PAN','')
                portion.append(sent.replace('PORTION:',''))
                    
            elif search('TEMPERATURE:', sent):
                temperature.append(sent.replace('TEMPERATURE:',''))
                
            elif search('PAN SIZE:', sent):
                pan_size.append(sent.replace('PAN SIZE:',''))
                    
            elif search('NOTE:', sent):
                note.append(sent.replace('NOTE:',''))
            else:
                method.append(sent)
        #merge method (as a list) into one single string
        method_str = ""
        for text in method:
            method_str += str(text) + " "
        savemethodstr = method_str
        
        for ingr in ingredients:
            if ingr.lower() in savemethodstr:
                #replace with ingredient mask
                savemethodstr = savemethodstr.replace(ingr.lower(), " INGREDIENT ")
                contains.append(ingr.lower())
                #turn ingredient into one token
                ingr_dash = ingr.lower().replace(' ', '')
                method_str = method_str.replace(ingr.lower(), ingr_dash)
                
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        method_str = tokenizer.tokenize(method_str)
        #lemmatize and POS tag the method string
        #PoS-Tag, lemmatize and format
        
        postagged = pos_tag_db(method_str)
        #lemmatized = lemmatize_db(postagged, [".", "X"])
        formatted = hp.formatFile(postagged)
        #method_tagged = formatted
        method_tagged = postagged
        
        
        onelist = []
        twolist = []
        contains_nospace = []
        for con in contains:
            cons = con.lower().replace(' ', '')
            contains_nospace.append(cons)
            
        for pair in method_tagged:
            onelist.extend(pair)
            
        for w, tag in onelist:
            pair = tuple([w, tag])
            if w in contains_nospace:
                pair2 = tuple((w, 'INGREDIENT'))
                twolist.append(pair2) 
            else:
                twolist.append(pair)
                
                        
        #the ingredients are turned into a set, so there is no repetition of ingredients.
        writer.writerow([portion, temperature, savemethodstr ,twolist, note, contains, set(contains)])   

## Third database: data/ArmedForcesRecipes.exl'

In [23]:
clean_db = hp.cleanFile(dbs[2])
armedforces_recipes = []
for sentence in clean_db:
    armedforces_recipes.append(sentence)
#split the recipes into lists
split_armedforces_recipes = str(armedforces_recipes).split('EACH')
del split_armedforces_recipes[0]

In [24]:
#writing the recipes to a csv file
with open('recipes_armed.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["EACH PORTION", 'TEMPERATURE', "METHODstr", "METHOD", "NOTE", "CONTAINS (NOUNS)", "INGREDIENTS"])
    #loop through each recipes 
    allmethod = []
    for recipe in split_armedforces_recipes:
        #for each recipe create a list
        portion = []
        method = []
        temperature = []
        pan_size = []
        note = []
        contains = []
        for sent in re.split(',|\n', recipe):
            if search('PORTION:', sent):
                sent.replace('PORTION:','')
                sent.replace('PAN','')
                portion.append(sent.replace('PORTION:',''))
                    
            elif search('TEMPERATURE:', sent):
                temperature.append(sent.replace('TEMPERATURE:',''))
                
            elif search('PAN SIZE:', sent):
                pan_size.append(sent.replace('PAN SIZE:',''))
                    
            elif search('NOTE:', sent):
                note.append(sent.replace('NOTE:',''))
            else:
                method.append(sent)
        #merge method (as a list) into one single string
        method_str = ""
        for text in method:
            method_str += str(text) + " "
        savemethodstr = method_str
        
        for ingr in ingredients:
            if ingr.lower() in savemethodstr:
                #replace with ingredient mask
                savemethodstr = savemethodstr.replace(ingr.lower(), " INGREDIENT ")
                contains.append(ingr.lower())
                #turn ingredient into one token
                ingr_dash = ingr.lower().replace(' ', '')
                method_str = method_str.replace(ingr.lower(), ingr_dash)
                
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        method_str = tokenizer.tokenize(method_str)
        #lemmatize and POS tag the method string
        #PoS-Tag, lemmatize and format
      
        postagged = pos_tag_db(method_str)
        #lemmatized = lemmatize_db(postagged, [".", "X"])
        formatted = hp.formatFile(postagged)
        #method_tagged = formatted
        method_tagged = postagged
        
        onelist = []
        twolist = []
        contains_nospace = []
        for con in contains:
            cons = con.lower().replace(' ', '')
            contains_nospace.append(cons)
            
        for pair in method_tagged:
            onelist.extend(pair)
            
        for w, tag in onelist:
            pair = tuple([w, tag])
            if w in contains_nospace:
                pair2 = tuple((w, 'INGREDIENT'))
                twolist.append(pair2) 
            else:
                twolist.append(pair)
        #the ingredients are turned into a set, so there is no repetition of ingredients.
        writer.writerow([portion, temperature, savemethodstr ,twolist, note, contains, set(contains)])   

## Fourth database: 'data/CommonRecipes.exl'

In [25]:
clean_db = hp.cleanFile(dbs[3])
common_recipes = []
for sentence in clean_db:
    common_recipes.append(sentence)
#split the recipes into lists
split_common_recipes = str(common_recipes).split('EACH')

In [26]:
#writing the recipes to a csv file
with open('recipes_common.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["EACH PORTION", 'TEMPERATURE', "METHODstr", "METHOD", "NOTE", "CONTAINS (NOUNS)", "INGREDIENTS"])
    #loop through each recipes 
    allmethod = []
    for recipe in split_common_recipes:
        #for each recipe create a list
        portion = []
        method = []
        temperature = []
        pan_size = []
        note = []
        contains = []
        for sent in re.split(',|\n', recipe):
            if search('PORTION:', sent):
                sent.replace('PORTION:','')
                sent.replace('PAN','')
                portion.append(sent.replace('PORTION:',''))
                    
            elif search('TEMPERATURE:', sent):
                temperature.append(sent.replace('TEMPERATURE:',''))
                
            elif search('PAN SIZE:', sent):
                pan_size.append(sent.replace('PAN SIZE:',''))
                    
            elif search('NOTE:', sent):
                note.append(sent.replace('NOTE:',''))
            else:
                method.append(sent)
        #merge method (as a list) into one single string
        method_str = ""
        for text in method:
            method_str += str(text) + " "
        savemethodstr = method_str
        
        for ingr in ingredients:
            if ingr.lower() in savemethodstr:
                #replace with ingredient mask
                savemethodstr = savemethodstr.replace(ingr.lower(), " INGREDIENT ")
                contains.append(ingr.lower())
                #turn ingredient into one token
                ingr_dash = ingr.lower().replace(' ', '')
                method_str = method_str.replace(ingr.lower(), ingr_dash)
                
        tokenizer = nltk.RegexpTokenizer(r"\w+")
        method_str = tokenizer.tokenize(method_str)
        #lemmatize and POS tag the method string
        #PoS-Tag, lemmatize and format
        
    
        postagged = pos_tag_db(method_str)
        #lemmatized = lemmatize_db(postagged, [".", "X"])
        formatted = hp.formatFile(postagged)
        #method_tagged = formatted
        method_tagged = postagged
        
        
        onelist = []
        twolist = []
        contains_nospace = []
        for con in contains:
            cons = con.lower().replace(' ', '')
            contains_nospace.append(cons)
            
        for pair in method_tagged:
            onelist.extend(pair)
            
        for w, tag in onelist:
            pair = tuple([w, tag])
            if w in contains_nospace:
                pair2 = tuple((w, 'INGREDIENT'))
                twolist.append(pair2) 
            else:
                twolist.append(pair)
                
            
        #the ingredients are turned into a set, so there is no repetition of ingredients.
        writer.writerow([portion, temperature, savemethodstr ,twolist, note, contains, set(contains)])   